In [1]:
import pandas as pd
import altair as alt
import numpy as np
from pathlib import Path
import re

In [2]:
root_path = Path('Data/London_dataset/csv/data_collection/data_tables')
result_path = Path('Data/London_dataset/DataResult')
preprocessed_path = Path('Data/London_dataset/preprocessed')
result_path.mkdir(exist_ok = True)
preprocessed_path.mkdir(exist_ok = True)

## Data_df

In [3]:
new_index = pd.date_range('2012-01-01 00:00', '2012-12-31 23:30', freq = '30min')
data_df = (
    # read all data files and concat
    pd.concat((pd.read_csv(path, index_col = 0) for path in root_path.glob('consumption*')), axis = 1)
    
    # make index datetime 
    .pipe(lambda x: x.set_axis(pd.to_datetime(x.index), axis = 0))

)

raw_data_df = (
    # start from the data df
    data_df
    
    # transpose 
    .T 
    
    # name axis 
    .sort_index()
    .rename_axis('meterID', axis = 0)
    .rename_axis('timestamp', axis = 1)
    
    # drop the incomplete days at the start and end
    .loc[:, '2013-01-01':'2013-12-31 23:30']
)

raw_data_df.to_csv("Data/London_dataset/csv/data_collection/data_tables/2013.csv", index=True)
print("done")
#display(raw_data_df.head())
#raw_data_df.to_pickle(result_path/'raw_data_df.pkl')
# multiindex_counts = raw_data_df.index.get_level_values('meterID').value_counts()
# count = 0
# for meterID in multiindex_counts.index:
#     if multiindex_counts[meterID] < 365 and multiindex_counts[meterID] > 350:
#         count += 1
#         print(meterID, multiindex_counts[meterID])
# print(f"count {count}")


done


In [4]:
raw_data_df.head()

timestamp,2013-01-01 00:00:00,2013-01-01 00:30:00,2013-01-01 01:00:00,2013-01-01 01:30:00,2013-01-01 02:00:00,2013-01-01 02:30:00,2013-01-01 03:00:00,2013-01-01 03:30:00,2013-01-01 04:00:00,2013-01-01 04:30:00,...,2013-12-31 19:00:00,2013-12-31 19:30:00,2013-12-31 20:00:00,2013-12-31 20:30:00,2013-12-31 21:00:00,2013-12-31 21:30:00,2013-12-31 22:00:00,2013-12-31 22:30:00,2013-12-31 23:00:00,2013-12-31 23:30:00
meterID,,,,,,,,,,,,,,,,,,,,,
D0000,1.043,0.404,0.185,0.151,0.139,0.105,0.122,0.091,0.110,0.100,...,0.247,0.323,0.346,0.238,0.335,0.144,0.234,0.182,0.153,0.166
D0001,0.181,0.248,0.206,0.171,0.068,0.083,0.083,0.072,0.060,0.061,...,0.265,0.165,0.179,0.163,0.145,0.227,0.256,0.117,0.078,0.025
D0002,0.224,0.227,0.231,0.236,0.239,0.251,0.246,0.334,0.287,0.292,...,0.156,0.173,0.164,0.154,0.163,0.176,0.145,0.177,0.158,0.162
D0003,0.077,0.078,0.076,0.077,0.077,0.077,0.077,0.076,0.079,0.077,...,0.092,0.093,0.094,0.093,0.093,0.093,0.094,0.090,0.055,0.049
D0004,0.195,0.202,0.188,0.039,0.024,0.102,0.033,0.024,0.033,0.028,...,0.006,0.020,0.016,0.022,0.011,0.026,0.006,0.006,0.025,0.012


In [5]:
nan_count_per_row = raw_data_df.isna().sum(axis=1)
count_missing = 0
count_ok = 0
for index, value in nan_count_per_row.items():
    if value > 10:
        print(f"index {index} has {value} nan values")
        count_missing += 1
    else:
        count_ok += 1

index D0001 has 50 nan values
index D0002 has 13 nan values
index D0025 has 49 nan values
index D0029 has 14 nan values
index D0038 has 53 nan values
index D0041 has 10463 nan values
index D0045 has 145 nan values
index D0048 has 24 nan values
index D0053 has 241 nan values
index D0064 has 53 nan values
index D0066 has 337 nan values
index D0076 has 49 nan values
index D0080 has 19 nan values
index D0087 has 13 nan values
index D0093 has 4463 nan values
index D0094 has 15 nan values
index D0096 has 34 nan values
index D0099 has 51 nan values
index D0102 has 14 nan values
index D0110 has 145 nan values
index D0118 has 13 nan values
index D0125 has 4468 nan values
index D0129 has 53 nan values
index D0134 has 1301 nan values
index D0143 has 289 nan values
index D0164 has 49 nan values
index D0167 has 338 nan values
index D0169 has 144 nan values
index D0188 has 14 nan values
index D0194 has 14 nan values
index D0195 has 21 nan values
index D0208 has 194 nan values
index D0212 has 1013 na

In [8]:
filtered_raw_data_df = raw_data_df[nan_count_per_row <= 10].copy()
filtered_raw_data_df.shape

(4276, 17520)

In [9]:
filtered_raw_data_df.interpolate(inplace=True)

In [10]:
filtered_raw_data_df.to_csv("Data/London_dataset/csv/data_collection/data_tables/2013_filtered_no_nan.csv", index=True)


In [46]:
filtered_raw_data_df.to_pickle(result_path/'2013_filtered_no_nan.pkl')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# Plot a histogram
plt.hist(filtered_raw_data_df, bins=range(5,filtered_raw_data_df.shape[1] + 1), edgecolor='black')
plt.xlabel('Number of Missing Values in a Row')
plt.ylabel('Number of Rows')
plt.title('Distribution of Missing Values in Rows')
plt.show()


In [ ]:
count_values = nan_count_per_row.value_counts()

# Plot a bar chart
count_values.plot(kind='bar', edgecolor='black')
plt.xlabel('Number of Missing Values in a Row')
plt.ylabel('Number of Rows')
plt.title('Distribution of Missing Values in Rows')
plt.show()

In [ ]:
count = 0
for meterID in multiindex_counts.index:
    if multiindex_counts[meterID] < 365 and multiindex_counts[meterID] > 350:
        count += 1
        print(meterID, multiindex_counts[meterID])
print(f"count {count}")

# Read info df and give columns names

In [11]:
questions = pd.read_csv(root_path/'survey_questions.csv', index_col = 0).drop(columns = 'Survey')
questions.head()

,Question
Question_id,
Q01,Q1 - Our household DOES NOT HAVE this appliance
Q02,"Q1 - For this appliance, our household managed..."
Q03,"Q1 - For this appliance, our household managed..."
Q04,"Q1 - For this appliance, our household managed..."
Q05,"Q1 - For this appliance, our household managed..."


In [13]:
info_df = (
    # read the survey answers
    pd.read_csv(root_path/'survey_answers.csv', low_memory = False, index_col = 0)
    
    # select appliance questionaire
    .loc[:, 'Q211':]
    
    # give the columns a better name based on the question dataframe
    .stack()
    .to_frame('answer')
    .rename_axis(index = ['meterID', 'Question_id'])
    .join(questions.shift(-1))
    .reset_index()
    .assign(
        question = lambda x: x['Question_id'].str.cat(x['Question'], sep = ': ')
    )
    .set_index(['meterID', 'question'])
    .drop(columns = ['Question_id', 'Question'])
    .unstack()
    .droplevel(0, axis = 1)
    .iloc[:, 1:]
)

info_df.head()

question,Q211: Smart Meter,Q212: Energy Decision Maker,Q213: Household Size,Q214: Household Member 1 Gender,Q215: Household Member 2 Gender,Q216: Household Member 3 Gender,Q217: Household Member 4 Gender,Q218: Household Member 5 Gender,Q219: Household Member 6 Gender,Q220: Household Member 7 Gender,...,Q334: Concern about climate change,Q335: Feel about current lifestyle and the environment,Q336: Bill since smrt meter was installed,Q337: Describe amount charged on most recent bill,Q338: Confident bill is accurate,Q339: How helpful is smart meter in understanding bill,Q340: How helpful is smart meter in understanding consumption,Q341: How helpful is smart meter in managingconsumption,Q342: Change in perception of EDF Energy,Q343: Likelihood to recommend
meterID,,,,,,,,,,,,,,,,,,,,,
D0000,Yes,Yes,2.0,Female,Male,NaN,NaN,NaN,NaN,NaN,...,Very concerned,I'd like to do a bit more to help the environment,Yes and it covered partly or fully the period ...,A little higher than I expected,don't know,Have not thought about it,Have not thought about it,Have not thought about it,Neither more nor less positive,9
D0001,Yes,Yes,1.0,Female,NaN,NaN,NaN,NaN,NaN,NaN,...,Fairly concerned,I'm happy with what I do at the moment,Yes and it covered partly or fully the period ...,About the amount I expected,Fairly confident,Fairly helpful,Fairly helpful,Fairly helpful,Neither more nor less positive,6
D0002,Yes,Yes,5.0,Male,Female,Male,Male,Female,NaN,NaN,...,Not very concerned,I'm happy with what I do at the moment,don't know,A little higher than I expected,don't know,Fairly helpful,Fairly helpful,Fairly helpful,Bit more positive,5
D0003,Yes,Yes,1.0,Male,NaN,NaN,NaN,NaN,NaN,NaN,...,Very concerned,I'd like to do a bit more to help the environment,Yes and it covered partly or fully the period ...,About the amount I expected,Fairly confident,Very helpful,Very helpful,Very helpful,Bit more positive,8
D0004,Yes,Yes,4.0,Female,Male,Male,Male,NaN,NaN,NaN,...,Very concerned,I'd like to do a bit more to help the environment,Yes but it covered the period before the smart...,About the amount I expected,Fairly confident,Very helpful,Very helpful,Fairly helpful,Bit more positive,7


# Preprocess questionair into features

In [14]:
# store all preprocessed features here 
features = pd.DataFrame(index = info_df.index)

## Features to keep and just rename

In [15]:
APPLIANCE_COLUMNS =['Q279: No. Fridges: type 1', 'Q280: No. Fridges: type 2',
       'Q281: No. Fridges: type 3', 'Q282: No. Fridges: type 4',
       'Q283: No. Freezers: type 1', 'Q284: No. Freezers: type 2',
       'Q285: No. Freezers: type 3', 'Q286: No. Freezers: type 4',
       'Q287: No. Freezers: type 5', 'Q288: No. Freezers: type 6',
       'Q289: No. Fridge-Freezers: type 1',
       'Q290: No. Fridge-Freezers: type 2',
       'Q291: No. Fridge-Freezers: type 3',
       'Q292: No. Fridge-Freezers: type 4', 'Q293: No. Electric hob',
       'Q294: No. Gas hob', 'Q295: No. Electric oven', 'Q296: No. Microwave',
       'Q297: No. Washing machine (not with dryer)', 'Q298: No. Tumble dryer',
       'Q299: No. Washer-dryer', 'Q300: No. Dishwasher',
       'Q301: No. Electric shower',
       'Q302: No. Over-sink electric water heater',
       'Q303: No. Portable electric heater', 'Q304: No. Television',
       'Q305: No. Desktop PC/computer', 'Q306: No. Laptop computer',
       'Q307: No. Printer', 'Q308: No. Router (for broadband internet)',
       'Q309: No. Dvd/blu-ray player', 'Q310: No. Cable TV box (e.g., Virgin)',
       'Q311: No. Satellite TV box (E.g., Sky)', 'Q312: No. Freeview TV box',
       'Q313: No. Games console', 'Q314: No. Standby savers']
appliance_rename = {key: key[10:] for key in APPLIANCE_COLUMNS}

TV_COLUMNS = [ 'Q315: TV Type 1', 'Q316: TV Type 2', 'Q317: TV Type 3',
       'Q318: TV Type 4', 'Q319: TV Type 5', 'Q320: TV Type 6']
tv_rename = {key: key[6:] for key in TV_COLUMNS} 

manual_rename =  {
    'Q213: Household Size': 'nb_of_inhabitants', 
    'Q234: Work from home': 'work_from_home', 
    'Q238: Rooms in home': 'nb_of_rooms',
    'Q239: Bedrooms': 'nb_of_bedrooms', 
    'Q240: Insulation: Double glazing': 'insulation_double_glazing', 
    'Q241: Insulation: Roof or loft insulation': 'insulation_roof', 
    'Q242: Insulation: Wall insulation': 'insulation_walls',
    'Q243: Insulation: Floor insulation': 'insulation_floor',
    'Q244: Insulation: Hot water tank insulation/lagging': 'insulation_warm_water_tank', 
}

appliance_rename = {**appliance_rename, **tv_rename, **manual_rename}

new_features = info_df[appliance_rename.keys()].rename(columns = appliance_rename)
print('features kept without preprocessing:')
with pd.option_context('display.max_rows', None, 'display.max_columns', 2) : 
    display(new_features.T)

features = features.join(new_features)

info_df = info_df.drop(columns = appliance_rename.keys())

features kept without preprocessing:


meterID,D0000,...,N4172
question,,,
Fridges: type 1,0.0,...,0.0
Fridges: type 2,0.0,...,0.0
Fridges: type 3,0.0,...,0.0
Fridges: type 4,0.0,...,0.0
Freezers: type 1,0.0,...,0.0
Freezers: type 2,0.0,...,0.0
Freezers: type 3,0.0,...,0.0
Freezers: type 4,0.0,...,0.0
Freezers: type 5,0.0,...,0.0


## Number of males and females

In [16]:
GENDER_COLUMNS = ['Q214: Household Member 1 Gender', 'Q215: Household Member 2  Gender',
       'Q216: Household Member 3 Gender', 'Q217: Household Member 4 Gender',
       'Q218: Household Member 5 Gender', 'Q219: Household Member 6 Gender',
       'Q220: Household Member 7 Gender', 'Q221: Household Member 8 Gender']


gender_info = info_df[GENDER_COLUMNS]
print('original')
display(gender_info.head())

features['nb_of_males'] = (gender_info == 'Male').sum(axis = 1)
features['nb_of_females'] = (gender_info == 'Female').sum(axis = 1)

print('derived features')
display(features[['nb_of_males', 'nb_of_females']].head())

info_df = info_df.drop(columns = GENDER_COLUMNS)

original


question,Q214: Household Member 1 Gender,Q215: Household Member 2 Gender,Q216: Household Member 3 Gender,Q217: Household Member 4 Gender,Q218: Household Member 5 Gender,Q219: Household Member 6 Gender,Q220: Household Member 7 Gender,Q221: Household Member 8 Gender
meterID,,,,,,,,
D0000,Female,Male,NaN,NaN,NaN,NaN,NaN,NaN
D0001,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D0002,Male,Female,Male,Male,Female,NaN,NaN,NaN
D0003,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D0004,Female,Male,Male,Male,NaN,NaN,NaN,NaN


derived features


,nb_of_males,nb_of_females
meterID,,
D0000,1,1
D0001,0,1
D0002,3,2
D0003,1,0
D0004,3,1


## Min, max and mean inhabitant age

In [17]:
AGE_COLUMNS = ['Q222: Household Member 1 Age', 'Q223: Household Member 2  Age',
       'Q224: Household Member 3 Age', 'Q225: Household Member 4 Age',
       'Q226: Household Member 5 Age', 'Q227: Household Member 6 Age',
       'Q228: Household Member 7 Age', 'Q229: Household Member 8 Age']

age = (
    info_df.loc[:, AGE_COLUMNS]
)

print('original')
display(age.head())

ages = age.stack().unique()
ages.sort()
replace_dict = {
    "Don't know": np.NAN
}
for key in ages[:-1]: 
    result = re.findall(r'\d+', key)
    replace_dict[key] = np.array(result).astype(float).mean()
replace_dict

age_features = (
    age.replace(replace_dict)
    .agg(['mean', 'min', 'max'], axis = 1)
    .set_axis(['mean_age', 'min_age', 'max_age'], axis = 1)
    # fill missing with mean of column
    .pipe(lambda df: df.fillna(df.mean(axis = 0)))
)

display('derived features')
display(age_features)

# add the the features dataframe
features = features.join(age_features)

info_df = info_df.drop(columns = AGE_COLUMNS)

original


question,Q222: Household Member 1 Age,Q223: Household Member 2 Age,Q224: Household Member 3 Age,Q225: Household Member 4 Age,Q226: Household Member 5 Age,Q227: Household Member 6 Age,Q228: Household Member 7 Age,Q229: Household Member 8 Age
meterID,,,,,,,,
D0000,35-44,12-15,NaN,NaN,NaN,NaN,NaN,NaN
D0001,65-74,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D0002,45-54,35-44,05-11,05-11,05-11,NaN,NaN,NaN
D0003,65-74,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D0004,45-54,45-54,12-15,18-24,NaN,NaN,NaN,NaN


'derived features'

,mean_age,min_age,max_age
meterID,,,
D0000,26.500000,13.5,39.5
D0001,69.500000,69.5,69.5
D0002,22.600000,8.0,49.5
D0003,69.500000,69.5,69.5
D0004,33.375000,13.5,49.5
...,...,...,...
N4166,69.500000,69.5,69.5
N4168,49.500000,49.5,49.5
N4169,75.000000,75.0,75.0


## Home ownership feature

In [18]:
home_ownership_replace_dict = {
    'Rents (with or without housing benefit) - from private landlord' : 'Rents' ,
    'Owns outright' : 'Owns' ,
    'Rents (with or without housing benefit) - from local authority/Council or Housing Association' : 'Rents' ,
    'Part owns and part rents (shared ownership)' : 'Shared ownership' ,
    'Owns with a mortgage or loan' : 'Owns' ,
    'Other' : 'Other' ,
    'Lives here rent-free' : 'Rent-free' ,
    'Other Other: LEASEHOLDER' : 'Rents' ,
    'Part owns and part rents (shared ownership) Other: HOUSING ASSOCIATION' : 'Rents' ,
    'Other Other: comes with job' : 'Rents' ,
    'Rents (with or without housing benefit) - from private landlord Other: SHELTERED ACCOMMODATION' : 'Rents' ,
    'Rents (with or without housing benefit) - from local authority/Council or Housing Association Other: SHARED OWNERSHIP' : 'Shared ownership' ,
    'Other Other: LEASE HOLDER' : 'Rents' ,
    'Other: RESIDENTIAL HOME' : 'Rents' ,
    'Other Other: COUNCIL' : 'Rents' ,
    'Other Other: RENTS FROM HM FORCES' : 'Rents' ,
    'Other Other: LEASEHOLD/OWNERSHIP' : 'Rents' ,
    'Other: LEASEHOLD' : 'Rents' ,
    'Other Other: SHELTERED HOUSING' : 'Rents' ,
    'Other Other: Rather not say' : 'Missing' ,
    'Owns outright Other: SHELTERED ACCOMMODATION' : 'Rents' ,
    'Other Other: RENT FROM LOCAL HOUSING CO-OPERATIVE' : 'Rents' ,
    'Part owns and part rents (shared ownership) Other: SHELTERED HOUSING' : 'Rents' ,
}

print('before preprocessing')
display(info_df['Q231: Newspapers - Printed'].value_counts(dropna=False).to_frame('count'))


# weird error in the column naming 
features['home_ownership'] = info_df['Q231: Newspapers - Printed'].replace(home_ownership_replace_dict)

print('after preprocessing') 
display(features['home_ownership'].value_counts(dropna = False).to_frame('count'))

info_df = info_df.drop(columns = 'Q231: Newspapers - Printed')

before preprocessing


,count
Q231: Newspapers - Printed,
Rents (with or without housing benefit) - from private landlord,1280
Owns outright,481
Rents (with or without housing benefit) - from local authority/Council or Housing Association,479
Part owns and part rents (shared ownership),181
Owns with a mortgage or loan,85
NaN,77
Other,6
Lives here rent-free,5
Other Other: LEASEHOLDER,4


after preprocessing


,count
home_ownership,
Rents,1776
Owns,566
Shared ownership,182
NaN,77
Other,6
Rent-free,5
Missing,1


## Building type

In [19]:
house_replacement = {
    'Terraced - middle': 'house-terraced', 
    'Terraced - end': 'house-semi-detached', 
    'Semi-detached': 'house-semi-detached', 
    'Detached': 'house-detached'
}

flat_replacement = {
    'In a purpose built block or tenement': 'flat',
	'Part of a converted house/building': 'flat',
	'In a commercial building, (for example in an office building, hotel, or over a shop)': 'flat',
}

mobile_replacement = {
    'A caravan or other mobile or temporary structure': 'mobile',
}

BUILDING_COLUMNS = ['Q235: Accomodation: House', 'Q236: Accomodation: Flat', 'Q237: Accomodation: Mobile']

print('before preprocessing')
display(info_df[BUILDING_COLUMNS])

building_type = (
    info_df[BUILDING_COLUMNS]
    .set_axis(['house', 'flat', 'mobile'], axis = 1)
    .replace(house_replacement)
    .replace(flat_replacement)
    .replace(mobile_replacement)
)
# fill in the correct values (some people filled some things twice)
building_type.loc[lambda x: ~x.flat.isna(), 'house'] = building_type.loc[lambda x: ~x.flat.isna(), 'flat']
building_type.loc[lambda x: ~x.mobile.isna(), 'house'] = building_type.loc[lambda x: ~x.mobile.isna(), 'mobile']

building_type_feature = building_type.house.to_frame('building_type')
building_type_feature

print('after preprocessing')
display(building_type_feature) 

features = features.join(building_type_feature)

info_df = info_df.drop(columns = BUILDING_COLUMNS)

before preprocessing


question,Q235: Accomodation: House,Q236: Accomodation: Flat,Q237: Accomodation: Mobile
meterID,,,
D0000,NaN,In a purpose built block or tenement,NaN
D0001,NaN,In a purpose built block or tenement,NaN
D0002,Semi-detached,NaN,NaN
D0003,NaN,In a purpose built block or tenement,NaN
D0004,Terraced - middle,NaN,NaN
...,...,...,...
N4166,NaN,In a purpose built block or tenement,NaN
N4168,Semi-detached,NaN,NaN
N4169,Terraced - middle,NaN,NaN


after preprocessing


,building_type
meterID,
D0000,flat
D0001,flat
D0002,house-semi-detached
D0003,flat
D0004,house-terraced
...,...
N4166,flat
N4168,house-semi-detached
N4169,house-terraced


## Central heating type

In [20]:
raw_heating = info_df['Q246: Central heating']

print('before_preprocessing')
display(raw_heating.value_counts(dropna = False).to_frame('count'))

central_heating_features = (
    pd.DataFrame(index = info_df.index, columns = ['gas', 'electric', 'other', 'solid', 'oil'])
    .assign(
        gas = raw_heating.str.contains('Gas'), 
        electric = raw_heating.str.contains('Electric'), 
        other = raw_heating.str.contains('Other central heating'), 
        solid = raw_heating.str.contains('Solid fuel'), 
        oil = raw_heating.str.contains('Oil')
    )
    .pipe(lambda x: x.set_axis('heating_'+x.columns, axis = 1))
)
print('after preprocessing')
display(central_heating_features)

features = features.join(central_heating_features)

info_df = info_df.drop(columns = 'Q246: Central heating')

before_preprocessing


,count
Q246: Central heating,
Gas,2113
No central heating,128
Electric (including storage heaters),114
Other central heating,81
Don't know,45
Gas;Electric (including storage heaters),45
NaN,25
Oil,16
No central heating;Electric (including storage heaters),13


after preprocessing


,heating_gas,heating_electric,heating_other,heating_solid,heating_oil
meterID,,,,,
D0000,True,False,False,False,False
D0001,True,False,False,False,False
D0002,True,False,False,False,False
D0003,False,False,True,False,False
D0004,True,False,False,False,False
...,...,...,...,...,...
N4166,False,False,False,False,False
N4168,True,False,False,False,False
N4169,True,False,False,False,False


## Heating control features

In [21]:
# if string contains key, replace with value
replacement_dict = {
    'switches on and off automatically at set times': 'heating_control_automatic_timed', 
    'manually at the boiler when needed': 'heating_control_manual_boiler',
    'automatically by a thermostatic': 'heating_control_automatic_temperature', 
    'control room temperature using thermostatic valves on the radiators': 'heating_control_manual_valves', 
    "I'm not sure": 'heating_control_unknown',
    "nan": 'heating_control_unknown'
}


raw_heating_control = info_df.loc[:, 'Q247: Central heating - control'].astype('str')

print('before preprocessing') 
display(raw_heating_control.value_counts(dropna = False).to_frame('count')) 

heating_control_features = pd.DataFrame(index = info_df.index)

for key,value in replacement_dict.items(): 
    if value not in heating_control_features.columns:
        heating_control_features[value] = raw_heating_control.str.contains(key)
    else: 
        heating_control_features[value] = heating_control_features[value] | raw_heating_control.str.contains(key)
heating_control_features

features = features.join(heating_control_features)
print('after preprocessing')
display(heating_control_features) 

info_df = info_df.drop(columns ='Q247: Central heating - control')

before preprocessing


,count
Q247: Central heating - control,
The heating switches on and off automatically at set times of the day,553
I switch the heating on manually at the boiler when needed,389
The heating is controlled automatically by a thermostatic temperature control,341
The heating switches on and off automatically at set times of the day;The heating is controlled automatically by a thermostatic temperature control,226
I control the room temperature using the thermostatic valves on the radiators,203
I control the room temperature using the thermostatic valves on the radiators;The heating switches on and off automatically at set times of the day;The heating is controlled automatically by a thermostatic temperature control,192
nan,184
I control the room temperature using the thermostatic valves on the radiators;The heating switches on and off automatically at set times of the day,128
I switch the heating on manually at the boiler when needed;I control the room temperature using the thermostatic valves on the radiators,77


after preprocessing


,heating_control_automatic_timed,heating_control_manual_boiler,heating_control_automatic_temperature,heating_control_manual_valves,heating_control_unknown
meterID,,,,,
D0000,False,True,False,False,False
D0001,False,False,True,False,False
D0002,False,True,False,False,False
D0003,False,True,False,False,False
D0004,True,True,True,False,False
...,...,...,...,...,...
N4166,False,False,False,False,True
N4168,False,False,True,False,False
N4169,False,True,False,False,False


## Water heating

In [22]:
# if string contains key, fill in value
replacement_dict = {
    'Hot water storage tank with gas boiler - used for both central heating and hot water': 'gas_w_storage',
    "Gas boiler (without hot water storage tank) - used for both central heating and hot water ('combi' boiler)": 'gas', 
    "Hot water storage tank with electric immersion heater": 'electric_w_storage', 
    "Hot water storage tank with gas boiler - used for hot water only": 'gas_w_storage',
    "Gas boiler (without hot water storage tank) - used for hot water only": 'gas', 
    "Other": 'other', 
    "Don't know": np.nan, 
    np.nan: np.nan, 
}

print('before preprocessing') 
display(info_df['Q248: Heating water'].value_counts(dropna = False).to_frame('value'))

heating_water_features = (
    info_df['Q248: Heating water']
    .astype('str')
    # some literal matching 
    .mask(lambda x: x.str.contains('Hot water storage tank with gas boiler - used for both central heating and hot water'), 'gas_w_storage')
    .mask(lambda x: x.str.contains("Gas boiler (without hot water storage tank) - used for both central heating and hot water", regex = False), 'gas')
    .mask(lambda x: x.str.contains("Hot water storage tank with electric immersion heater"), 'electric_w_storage')
    .mask(lambda x: x.str.contains("Hot water storage tank with gas boiler - used for hot water only", regex = False), 'gas_w_storage')
    .mask(lambda x: x.str.contains("Gas boiler (without hot water storage tank) - used for hot water only", regex = False), 'gas')
    # if other but gas mentioned -> gas 
    .mask(lambda x: x.str.contains("Other") & x.str.lower().str.contains('gas'), 'gas')
    # if other but electric mentioned -> electric 
    .mask(lambda x: x.str.contains("Other") & x.str.lower().str.contains('electric'), 'electric_w_storage')
    # remaining others are NaN
    .mask(lambda x: x.str.contains("Don't know") | x.str.contains('Other') , np.nan)
    .to_frame('water_heating')
)
print('after_preprocessing') 
display(heating_water_features.value_counts().to_frame('count'))

features = features.join(heating_water_features) 
info_df = info_df.drop(columns = 'Q248: Heating water')

before preprocessing


,value
Q248: Heating water,
Hot water storage tank with gas boiler - used for both central heating and hot water,1057
Gas boiler (without hot water storage tank) - used for both central heating and hot water ('combi' boiler),902
Hot water storage tank with electric immersion heater,248
Hot water storage tank with gas boiler - used for hot water only,85
Gas boiler (without hot water storage tank) - used for hot water only,81
...,...
Other; Other: Centrally heated,1
Don't know; Other: POWER SHOWER AND KETTLE,1
Other; Other: communal oil heated,1


after_preprocessing


,count
water_heating,
gas_w_storage,1150
gas,992
electric_w_storage,261
nan,21


## Irrelevant columns
Everything that remains is 'irrelevant'

In [23]:

with pd.option_context('display.max_rows', None, 'display.max_columns', 2) : 
    display(info_df.T)

meterID,D0000,...,N4172
question,,,
Q211: Smart Meter,Yes,...,Yes
Q212: Energy Decision Maker,Yes,...,Yes
Q230: Relationship to Others in household,Child/children; Other: SON,...,Husband/wife/partner;Brother(s) / sister(s)
Q232: Newspapers - Online,NaN,...,NaN
Q233: Newspapers - None read regularly,NaN,...,NaN
Q245: Insulation: Other,NaN,...,NaN
Q249: No. Light bulbs: Living room: Traditional,0.0,...,0.0
Q250: No. Light bulbs: Kitchen: Traditional,0.0,...,0.0
Q251: No. Light bulbs: Dining room: Traditional,0.0,...,0.0


## preprocessed features

Overview of the preprocessed features

In [24]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 2) : 
    display(features.T)
info_df = features

meterID,D0000,...,N4172
Fridges: type 1,0.0,...,0.0
Fridges: type 2,0.0,...,0.0
Fridges: type 3,0.0,...,0.0
Fridges: type 4,0.0,...,0.0
Freezers: type 1,0.0,...,0.0
Freezers: type 2,0.0,...,0.0
Freezers: type 3,0.0,...,0.0
Freezers: type 4,0.0,...,0.0
Freezers: type 5,0.0,...,0.0
Freezers: type 6,0.0,...,0.0


In [25]:
info_df.to_pickle(result_path/'raw_info_df_features.pkl')

# Final preprocessing for the tree models

## Gender columns are OK

## Water heating, fill NaNs as value between electrical and gas 

In [26]:
print('before preprocessing') 
display(info_df.water_heating.value_counts(dropna = False))

info_df['water_heating'] = info_df['water_heating'].replace({
    'electric_w_storage': 1, 
    np.NaN: 1.5, 
    'nan': 1.5, 
    'gas':2, 
    'gas_w_storage': 3
})

print('after preprocessing')
info_df.water_heating.value_counts(dropna = False)

before preprocessing


water_heating
gas_w_storage         1150
gas                    992
electric_w_storage     261
NaN                    189
nan                     21
Name: count, dtype: int64

after preprocessing


water_heating
3.0    1150
2.0     992
1.0     261
1.5     210
Name: count, dtype: int64

## Fill NaN with means for nb_of_inhabitants, nb_of_rooms, nb_of_bedrooms

In [27]:
print('before preprocessing') 
display(info_df.nb_of_inhabitants.value_counts(dropna = False))

columns = ['nb_of_inhabitants', 'nb_of_rooms', 'nb_of_bedrooms']
info_df = info_df.fillna(info_df[columns].mean(axis = 0))
print('after preprocessing')
info_df.nb_of_inhabitants.value_counts(dropna = False)

before preprocessing


nb_of_inhabitants
1.0     916
2.0     914
3.0     306
4.0     193
NaN     174
5.0      75
6.0      20
7.0       6
9.0       3
8.0       3
11.0      2
12.0      1
Name: count, dtype: int64

after preprocessing


nb_of_inhabitants
1.000000     916
2.000000     914
3.000000     306
4.000000     193
2.072981     174
5.000000      75
6.000000      20
7.000000       6
9.000000       3
8.000000       3
11.000000      2
12.000000      1
Name: count, dtype: int64

## Home ownership: fill NaN's and encode as one_hot_encoding

In [28]:
print('before preprocessing') 
info_df.home_ownership.value_counts(dropna = False)

before preprocessing


home_ownership
Rents               1776
Owns                 566
Shared ownership     182
NaN                   77
Other                  6
Rent-free              5
Missing                1
Name: count, dtype: int64

In [29]:
info_df['home_ownership'] = info_df['home_ownership'].replace({np.NaN: 'Missing'}).str.lower().str.replace(' ', '_')
info_df.home_ownership.value_counts(dropna = False)

home_ownership_features = pd.get_dummies(info_df['home_ownership'], prefix = 'home_ownership')

info_df = (
    info_df
    .drop(columns = 'home_ownership')
    .join(home_ownership_features)

)

print('after preprocessing') 
home_ownership_features.head()

after preprocessing


,home_ownership_missing,home_ownership_other,home_ownership_owns,home_ownership_rent-free,home_ownership_rents,home_ownership_shared_ownership
meterID,,,,,,
D0000,False,False,True,False,False,False
D0001,False,False,False,False,True,False
D0002,False,False,True,False,False,False
D0003,False,False,True,False,False,False
D0004,False,False,False,False,True,False


## Work_from_home fill NaN's with -1 and encode as ordinal

In [30]:
print('before preprocessing')
info_df.work_from_home.value_counts(dropna = False)

before preprocessing


work_from_home
Never                  1831
Occasionally            418
Most/all weekdays       155
About half the time     110
NaN                      99
Name: count, dtype: int64

In [31]:
info_df['work_from_home'] = info_df['work_from_home'].replace({
    np.NaN: -1, 
    'Never': 0, 
    'Occasionally': 1, 
    'About half the time': 2, 
    'Most/all weekdays': 3
})

In [32]:
print('after preprocessing')
info_df.work_from_home.value_counts(dropna = False)

after preprocessing


work_from_home
 0    1831
 1     418
 3     155
 2     110
-1      99
Name: count, dtype: int64

## Building type to ordinal

In [33]:
print('before preprocessing') 
info_df.building_type.value_counts(dropna = False)

before preprocessing


building_type
flat                   922
house-semi-detached    766
house-terraced         705
house-detached         178
NaN                     38
mobile                   4
Name: count, dtype: int64

In [34]:
info_df['building_type'] = info_df['building_type'].replace({
    np.NaN: -1, 
    "mobile": 0, 
    "flat":1, 
    'house-terraced': 2, 
    'house-semi-detached':3, 
    'house-detached': 4
})

In [35]:
print('after preprocessing') 
info_df.building_type.value_counts(dropna = False)

after preprocessing


building_type
 1    922
 3    766
 2    705
 4    178
-1     38
 0      4
Name: count, dtype: int64

## TVs to count of TVs per type 

In [36]:
tv_type_columns = ['TV Type 1', 'TV Type 2', 'TV Type 3', 'TV Type 4',
       'TV Type 5', 'TV Type 6']

print('before preprocessing') 
info_df[tv_type_columns].head()

before preprocessing


,TV Type 1,TV Type 2,TV Type 3,TV Type 4,TV Type 5,TV Type 6
meterID,,,,,,
D0000,Plasma,LCD,NaN,NaN,NaN,NaN
D0001,LED,NaN,NaN,NaN,NaN,NaN
D0002,Plasma,LED,NaN,NaN,NaN,NaN
D0003,LCD,NaN,NaN,NaN,NaN,NaN
D0004,Plasma,Plasma,Traditional/older style (CRT),Traditional/older style (CRT),NaN,NaN


In [37]:
tv_counts = (
    info_df[tv_type_columns]
    .replace({"Don't know":'Unknown', 'Traditional/older style (CRT)':'crt'})
    .apply(lambda x: x.value_counts(),axis = 1)
    # fill NaN's with 0 
    .fillna(0)
    .rename(columns = lambda x: f'tv_count_{x.lower()}')
)


info_df = (
    info_df
    .drop(columns = tv_type_columns)
    .join(tv_counts)
)

print('after preprocessing') 
tv_counts.head()

after preprocessing


,tv_count_lcd,tv_count_led,tv_count_other,tv_count_plasma,tv_count_unknown,tv_count_crt
meterID,,,,,,
D0000,1.0,0.0,0.0,1.0,0.0,0.0
D0001,0.0,1.0,0.0,0.0,0.0,0.0
D0002,0.0,1.0,0.0,1.0,0.0,0.0
D0003,1.0,0.0,0.0,0.0,0.0,0.0
D0004,0.0,0.0,0.0,2.0,0.0,2.0


## Insulation to 0, 0.5, 1 (False, Missing, True)

In [38]:
columns = ['insulation_double_glazing', 'insulation_roof', 'insulation_walls', 'insulation_floor', 'insulation_warm_water_tank']

In [39]:
info_df[columns] = info_df[columns].replace({
    'Yes':1, 
    'No': 0, 
    np.NaN: 0.5, 
    "Don't know": 0.5, 
})

## Heating columns to 0, 0.5, 1 (False, Missing, True)

In [40]:
heating_columns = ['heating_gas',
       'heating_electric', 'heating_other', 'heating_solid', 'heating_oil',
       'heating_control_automatic_timed', 'heating_control_manual_boiler',
       'heating_control_automatic_temperature',
       'heating_control_manual_valves', 'heating_control_unknown']
info_df[heating_columns].stack(dropna = False).value_counts(dropna = False)

False    20378
True      5627
NaN        125
Name: count, dtype: int64

In [41]:
info_df[heating_columns] = info_df[heating_columns].replace({
    np.NaN: 0.5, 
    False: 0, 
    True: 1
})

## The fridge freezer and appliance columns are ok

In [42]:
fridge_columns = ['Fridges: type 1', 'Fridges: type 2', 'Fridges: type 3',
       'Fridges: type 4', 'Freezers: type 1', 'Freezers: type 2',
       'Freezers: type 3', 'Freezers: type 4', 'Freezers: type 5',
       'Freezers: type 6', 'Fridge-Freezers: type 1',
       'Fridge-Freezers: type 2', 'Fridge-Freezers: type 3',
       'Fridge-Freezers: type 4']
info_df[fridge_columns].stack().value_counts(dropna = False)

0.0     32576
1.0      3812
2.0       184
3.0         9
10.0        1
Name: count, dtype: int64

In [43]:
appliance_columns = ['Electric hob', 'Gas hob', 'Electric oven', 'Microwave',
       'Washing machine (not with dryer)', 'Tumble dryer', 'Washer-dryer',
       'Dishwasher', 'Electric shower', 'Over-sink electric water heater',
       'Portable electric heater', 'Television', 'Desktop PC/computer',
       'Laptop computer', 'Printer', 'Router (for broadband internet)',
       'Dvd/blu-ray player', 'Cable TV box (e.g., Virgin)',
       'Satellite TV box (E.g., Sky)', 'Freeview TV box', 'Games console',
       'Standby savers']
info_df[appliance_columns].stack().value_counts(dropna = False)

0.0     32566
1.0     21440
2.0      2361
3.0       685
4.0       308
5.0        75
6.0        33
7.0         6
8.0         3
12.0        2
21.0        2
10.0        1
11.0        1
63.0        1
41.0        1
9.0         1
Name: count, dtype: int64

## Final state

In [44]:
info_df.columns = info_df.columns.str.lower().str.replace(': ', '_').str.replace(' ', '_')
info_df = info_df.rename(columns = {'cable_tv_box_(e.g.,_virgin)': 'cable_tv_box', 'satellite_tv_box_(e.g.,_sky)': 'satellite_tv_box'})
with pd.option_context('display.max_rows', None,
                       'display.max_columns', 3):
    display(info_df.T)
info_df.to_pickle(result_path/'raw_info_df_numerical.pkl')

meterID,D0000,...,N4172
fridges_type_1,0.0,...,0.0
fridges_type_2,0.0,...,0.0
fridges_type_3,0.0,...,0.0
fridges_type_4,0.0,...,0.0
freezers_type_1,0.0,...,0.0
freezers_type_2,0.0,...,0.0
freezers_type_3,0.0,...,0.0
freezers_type_4,0.0,...,0.0
freezers_type_5,0.0,...,0.0
freezers_type_6,0.0,...,0.0


# Preprocess to final version

In [50]:
from util import transform_raw_data_and_save, check_dataset

yearly_data_df, daily_data_df, yearly_info_df, daily_info_df = transform_raw_data_and_save(
    raw_data_df = result_path/'2013_filtered_no_nan.pkl', 
    yearly_info_df = result_path/'raw_info_df_numerical.pkl', 
    result_path = preprocessed_path, 
    weather_city = 'London',
    holiday_country = 'England', 
    year_to_use_as_index = 2013
)
check_dataset((yearly_data_df, daily_data_df, yearly_info_df, daily_info_df))

In [53]:
def get_meterid_count(df):
    multiindex_counts = df.index.get_level_values('meterID').value_counts()
    count = 0
    for meterID in multiindex_counts.index:
        if multiindex_counts[meterID] < 365 and multiindex_counts[meterID] > 350:
            count += 1
            print(meterID, multiindex_counts[meterID])
    print(f"count {count}")

In [ ]:
raw = pd.read_pickle(result_path/'raw_data_df.pkl')
get_meterid_count(raw)
print("##############################################")
get_meterid_count(daily_info_df)

In [66]:
daily_data_df_filtered = daily_data_df.drop(index=daily_data_df.index.get_level_values("meterID") == 'D0000').copy()


KeyError: True

In [72]:
daily_data_df_filtered = daily_data_df[daily_data_df.index.get_level_values('meterID') != 'D0000']
yearly_data_df_filtered = yearly_data_df[yearly_data_df.index.get_level_values('meterID') != 'D0000']
yearly_info_df_filtered = yearly_info_df[yearly_data_df.index.get_level_values('meterID') != 'D0000']
daily_info_df_filtered = daily_info_df[daily_data_df.index.get_level_values('meterID') != 'D0000']

In [75]:
daily_data_df_filtered.to_pickle(preprocessed_path/'daily_data_df_filtered.pkl')
yearly_data_df_filtered.to_pickle(preprocessed_path/'yearly_data_df_filtered.pkl')
yearly_info_df_filtered.to_pickle(preprocessed_path/'yearly_info_df_filtered.pkl')
daily_info_df_filtered.to_pickle(preprocessed_path/'daily_info_df_filtered.pkl')

In [70]:
get_meterid_count(df_filtered)

count 0


In [ ]:
yearly_data_df.index.get_level_values(0)
#yearly_data_df.to_csv("Data/London_dataset/preprocessed/csv/yearly_info_df.csv",index=False)

In [ ]:
daily_info_df.tail()
daily_info_df.to_csv("Data/London_dataset/preprocessed/csv/daily_info_df.csv",index=True)

In [73]:

merged = daily_info_df_filtered.merge(yearly_info_df_filtered,on=['meterID', 'year'], how="left")
merged.to_pickle("Data/London_dataset/preprocessed/combined_attributes_filtered.pkl")